## 프로젝트 : 동영상 속 인물의 얼굴을 인식하여 캐릭터 씌우기

## Face Detection VS Face Recognization

## 아나콘다 프롬프트(또는 관리자 권한 실행)에 패키지 설치 (pip install mediapipe)

In [1]:
pip install mediapipe # mediapipe를 사용해야 사진 및 영상 속의 특정 인체 부분들을 감지할 수 있다.

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


In [8]:
import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay_image) :  # 대상 이미지, x, y 좌표, 폭, 높이, 덮어씌울 이미지
    # 대상 이미지가 3채널일 경우 덮어씌울 이미지는 4채널
    alpha = overlay_image[:, :, 3] # BGRA, 채널 변경
    mask_image = alpha / 255 
    # alpha의 값의 범위는 0 ~ 255로, 1에 가까울수록 불투명해지고 0에 가까울수록 형체가 완전해진다.
    # mask 이미지로 변형시키면
    # (255, 255) -> (1, 1)
    # (255, 0) -> (1, 0)

    # 1 - mask_image 는?
    # (1-1, 1-1) = (0, 0)
    # (1-1, 1-0) = (0, 1)
    # (255, x) -> (1, 0.3)이면 (0, 0.7)이 된다.

# 얼굴을 찾고, 찾은 얼굴에 표시를 해 주기 위한 변수를 정의한다.
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈을 사용한다.
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용한다.

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

# 이미지 불러오기(이 코드를 사용하고 싶다면 양 눈에 원 그리기 부분을 주석 처리할 것.)
# 그리고 4채널 중 필요한 만큼을 추출하여 3채널에 대입하는 과정에서 추가적인 연산을 수행할 함수를 만들어야 한다.
# 참고 : 특정 부분들마다 적용시킬 이미지의 픽셀 값을 숙지해야 한다.
image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED) # 100 X 100
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED) # 100 X 100
image_nose = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED) # 300 X 100 (가로, 세로)

with mp_face_detection.FaceDetection(model_selection = 0, min_detection_confidence = 0.7) as face_detection:
    # 영상 내의 사람의 얼굴이라는 최소한의 확신 기준인 min_detection_confidence 의 값이 1에 가까울수록 얼굴의 정면이 뚜렷한 부분만을 검출한다.
    # 즉, 확률이 아닌 신뢰도를 의미한다.
    while cap.isOpened() : # 열고자 하는 동영상 파일을 여는 데 성공하면
        success, image = cap.read() # 동영상 파일의 내용을 프로그램이 읽는다.
        if not success : # 열고자 하는 동영상 파일을 여는 데 성공하지 못할 경우
            break # 실행되지 않고 강제적으로 종료된다.

        # 선택적으로 이미지를 기준으로 전달할 수 없도록 표시하면 성능이 향상된다.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # 기본적으로 OpenCV는 이미지의 3원색 처리를 BGR로 하지만, # mediapipe는 COLOR_BGR2RGB로 인해 BGR로 처리했던 방식을 RGB로 처리한다.
        results = face_detection.process(image)

        # 이미지에 얼굴 감지 주석을 그린다.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # mediapipe를 활용한 처리를 위해 RGB로 변경되어 있었다가 BGR로 복원시킨다.
        if results.detections :
            # 검출된 얼굴이 있을 경우 검출된 인물의 얼굴만큼 반복문을 통하여 draw
            # 6개 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀
            for detection in results.detections :
                mp_drawing.draw_detection(image, detection)
                # print(detection) # 실행되는 동안의 6개 특징을 포함한 detection 정보 출력

                #특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] # 오른쪽 눈
                left_eye = keypoints[1] # 왼쪽 눈
                nose_tip = keypoints[2] # 코 끝부분

                h, w, _ = image.shape # height, width, channel : 이미지로부터 세로, 가로의 크기를 가져온다.
                right_eye = (int(right_eye.x * w), int(right_eye.y * h)) # 이미지 내의 실제 좌표(x, y), 좌표 값 변경이 없으면 사람의 얼굴 중 오른쪽 눈에 적용
                left_eye = (int(left_eye.x * w), int(left_eye.y * h)) # 이미지 내의 사람의 얼굴 중 왼쪽 눈의 실제 좌표(x, y), 좌표 값 변경이 없으면 사람의 얼굴 중 왼쪽 눈에 적용
                nose_tip = (int((nose_tip).x * w), int(nose_tip.y * h)) # 이미지 내의 사람의 얼굴 중 코 끝부분의 실제 좌표(x, y)

                # 양 눈에 원 그리기
                # cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA) # 오른쪽 눈에는 파란색 원
                # cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA) # 왼쪽 눈에는 초록색 원
                # 코에 원 그리기
                # cv2.circle(image, nose_tip, 75, (0, 0, 255), 10, cv2.LINE_AA) # 코에는 빨간색 원

                # 특정 부분에 이미지 그리기
                # image[right_eye[1]-50:right_eye[1]+50, right_eye[0]-50:right_eye[0]+50] = image_right_eye # 오른쪽 눈
                # image[left_eye[1]-50:left_eye[1]+50, left_eye[0]-50:left_eye[0]+50] = image_left_eye # 왼쪽 눈
                # image[nose_tip[1]-50:nose_tip[1]+50, nose_tip[0]-50:nose_tip[0]+150] = image_nose # 코

                # image, x, y, w, h, overlay_image(조금 더 간단하게 특정 부분에 이미지 그리기)
                overlay(image, *right_eye, 50, 50, image_right_eye) # 오른쪽 눈의 지점에서는 x축과 y축의 크기를 각각 50으로 변경
                overlay(image, *left_eye, 50, 50, image_left_eye) # 왼쪽 눈의 지점에서는 x축과 y축의 크기를 각각 50으로 변경
                overlay(image, *nose_tip, 150, 50, image_nose) # 코 지점에서는 x축의 크기를 150, y축의 크기를 50으로 변경
                # 이곳의 overlay 코드들에서는 사용하고자 하는 이미지의 픽셀 값의 반으로 적용시키면 된다.

        # 셀카 보기 방식으로 볼 수 있도록 이미지를 수평으로 뒤집기
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx = 0.5, fy=0.5)) 

        if cv2.waitKey(1) == ord('q'): 
            break # q를 누르면 종료한다.
            
cap.release() #의도한 과정들이 모두 처리된 후의 영상을 보인다.
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'mediapipe.python._framework_bindings'

In [3]:
image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED)
image_right_eye.shape
# 오른쪽 눈 부분에 적용할 이미지의 폭과 높이의 크기와 몇 채널인지 표시한다.

(100, 100, 3)

In [4]:
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED)
image_left_eye.shape
# 왼쪽 눈 부분에 적용할 이미지의 폭과 높이의 크기와 몇 채널인지 표시한다.

(100, 100, 3)

In [5]:
image_nose = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED)
image_nose.shape
# 코 부분에 적용할 이미지의 폭과 높이의 크기와 몇 채널인지 표시한다.

(100, 300, 3)